# PyTorch Lightning

We want to use PyTorch Lightning  now.

In [1]:
import os
from pathlib import Path
from typing import Optional
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchmetrics
import torchvision
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import lightning.pytorch as pl

from torch.utils.data import random_split, DataLoader
from torchvision.datasets.utils import download_and_extract_archive

## Datamodule

In [2]:
class CatDogImageDataModule(pl.LightningDataModule):
    def __init__(self, dl_path='./tmp', batch_size = 32, num_workers=0, cache_dataset=True):
        super().__init__()
        self._dl_path = dl_path
        self._batch_size = batch_size
        self._num_workers = num_workers
        self._cache_dataset = cache_dataset

    @property
    def data_path(self):
        return Path(self._dl_path).joinpath("PetImages")

    @property
    def normalize_transform(self):
        return transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        
    @property
    def train_transform(self):
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            self.normalize_transform,
        ])

    @property
    def val_transform(self):
        return transforms.Compose([
            transforms.Resize((224, 224)), 
            transforms.ToTensor(), 
            self.normalize_transform
        ])

    def prepare_data(self):
        """Download images and prepare images datasets."""
        url = 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip'  
        if len(os.listdir(self.data_path)) == 0:
            download_and_extract_archive(url=url, download_root=self._dl_path, remove_finished=not self._cache_dataset)
        else:
            print("Dataset already exists, skipping download and extraction...")

    def setup(self, stage: Optional[str] = None):
        # make assignments here (val/train/test split)
        dataset = self.create_dataset(self.data_path, self.train_transform)
        self.train_data, self.val_data = random_split(dataset, [0.8, 0.2])

        print("Dataset created, split:")
        print(f'training images: {len(self.train_data)}')
        print(f'validation images: {len(self.val_data)}')
        #self.log.info("Dataset created, split:")
        #log.info(f'training images: {len(self.train_data)}')
        #log.info(f'validation images: {len(self.val_data)}')

    def create_dataset(self, root, transform):
        return ImageFolder(root=root, transform=transform, is_valid_file=self._is_image_valid)

    def train_dataloader(self):
        #self.log.info("Training data loaded.")
        return DataLoader(dataset=self.train_data, batch_size=self._batch_size, num_workers=self._num_workers, shuffle=True)
    
    def val_dataloader(self):
        #log.info("Validation data loaded.")
        return DataLoader(dataset=self.val_data, batch_size=self._batch_size, num_workers=self._num_workers, shuffle=False)

    def _is_image_valid(self, image_path):
        try:
            image = Image.open(image_path)
            return True
        except:
            return False

In [3]:
dm = CatDogImageDataModule(num_workers=16) #

In [4]:
#dm.prepare_data()

In [5]:
#dm.setup()

## Build Lightning Model

In [6]:
class SimpleCNN(pl.LightningModule):
    def __init__(self):
        super().__init__()
        
        self.conv_1 = nn.Sequential(nn.Conv2d(3, 16, 3), nn.ReLU(), nn.MaxPool2d(2,2))
        self.conv_2 = nn.Sequential(nn.Conv2d(16, 32, 3), nn.ReLU(), nn.MaxPool2d(2,2))
        self.conv_3 = nn.Sequential(nn.Conv2d(32, 64, 3), nn.ReLU(), nn.MaxPool2d(2,2))
        self.fc_1 = nn.Sequential(nn.Flatten(), nn.Linear(43264,256), nn.ReLU(), nn.Linear(256,128), nn.ReLU())
        self.fc_2 = nn.Sequential(nn.Linear(128,2),)

        #mat1 and mat2 shapes cannot be multiplied (32x43264 and 1600x256)

    def cross_entropy_loss(self, logits, labels):
      return F.nll_loss(logits, labels)

    def training_step(self, batch, batch_idx):
        data, label = batch
        output = self.forward(data)
        loss = nn.CrossEntropyLoss()(output,label)
        self.log('train_loss', loss)
        return {'loss': loss, 'log': self.log}

    def validation_step(self, batch, batch_idx):
        val_data, val_label = batch
        val_output = self.forward(val_data)
        val_loss = nn.CrossEntropyLoss()(val_output, val_label)
        self.log('val_loss', val_loss)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

    def forward(self, x):
        x = self.conv_1(x)
        x = self.conv_2(x)
        x = self.conv_3(x)
        x = self.fc_1(x)
        x = self.fc_2(x)
        return F.softmax(x,dim = 1) 

In [7]:
if torch.cuda.is_available():
    torch.set_float32_matmul_precision('high')

In [8]:
model = SimpleCNN()

## Setup trainer

In [ ]:
trainer = pl.Trainer(max_epochs=20)

trainer.fit(model, datamodule=dm)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Dataset already exists, skipping download and extraction...


/home/user/mambaforge/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Dataset created, split:
training images: 19999
validation images: 4999


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params
--------------------------------------
0 | conv_1 | Sequential | 448   
1 | conv_2 | Sequential | 4.6 K 
2 | conv_3 | Sequential | 18.5 K
3 | fc_1   | Sequential | 11.1 M
4 | fc_2   | Sequential | 258   
--------------------------------------
11.1 M    Trainable params
0         Non-trainable params
11.1 M    Total params
44.530    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]